In [13]:
import requests
import pandas as pd
import numpy as np
import requests 
import os
from io import StringIO  # Import StringIO from the io module
import time  # Import the time module
from urllib.parse import quote  # To URL encode SMILES
import plotly.express as px
import matplotlib.colors as mcolors

Befor runing this code: 
1) create a .csv file containing a 'species_header' like 'species', 'names', etc. this column should contain the name of the species (or genuses by default) resolved by Open Tree of Life.
2) creta an empty folder (input_folder) and put the .csv file inside! 

That's all

DEFINE THE FOLLOWING PARAMETERS

In [14]:
#Paths
# Input folder containing CSV files
input_folder = '/mnt/c/Users/quirosgu/Desktop/OK/' #/home/quirosgu/Desktop/FARMA-SHARE/RECHERCHE/FASIE_LAB/LuisQ/V1V2/'

species_header = 'full_species'#'query_otol_species' #put the column of your species

LOTUSDB = '/mnt/c/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv' #'/home/quirosgu/Desktop/FARMA-SHARE/RECHERCHE/FASIE_LAB/LuisQ/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv'
LOTUSDB_rc = '/mnt/c/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_rc.csv' #'/home/quirosgu/Desktop/FARMA-SHARE/RECHERCHE/FASIE_LAB/LuisQ/Yggdrasil/data_loc/LotusDB_inhouse_rc.csv'

# Output folder for the processed CSV files
output_folder = f'{input_folder}output_data/'

In [17]:
import os
import pandas as pd

# List all CSV files in the folder
csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

# Loop through each CSV file
for file_name in csv_files:
    # Construct the full file path
    file_path = os.path.join(input_folder, file_name)
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Extract 'MassIVE' code from 'feature' column
    df['MassIVE'] = df['feature'].str.extract(r'mzspec:(MSV\d+):')

    # Extract 'ms_filename' from 'feature' column
    ms_filename_match = df['feature'].str.extract(r'[^:]+:[^:]+:([^:]+)_features_ms2_(pos|neg).mgf:')
    if ms_filename_match is not None and len(ms_filename_match.columns) == 2:
        df['ms_filename'] = ms_filename_match[0]
    
    # Extract 'scan' from 'feature' column
    df['scan'] = df['feature'].str.extract(r'scan:(\d+)')
    
    # Perform any additional modifications to the DataFrame here
    
    # Save the modified DataFrame back to the same file
    df.to_csv(file_path, index=False)  # Set index=False to exclude the index column


In [22]:
# List all CSV files in the folder
csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

# Initialize an empty list to store DataFrames
dfs = []

# Loop through each CSV file
for file_name in csv_files:
    # Construct the full file path
    file_path = os.path.join(input_folder, file_name)
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Filter rows where 'jaccard_similarity' > 0.1
    filtered_df = df[df['jaccard_similarity'] > 0.25]
    
    # Extract unique 'feature_query' and 'ms_filename' pairs
    unique_pairs = filtered_df[['feature_query', 'ms_filename']].drop_duplicates()
    
    # Append the unique pairs to the list of DataFrames
    dfs.append(unique_pairs)

# Concatenate all DataFrames into a single combined DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Save the combined DataFrame to a CSV file
combined_df.to_csv(f'{input_folder} combined_results_js25.csv', index=False)
